# Cat vs Dog CNN Classification using TensorFlow/Keras

In [ ]:
import zipfile
import tensorflow as tf
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, BatchNormalization, Dropout
import matplotlib.pyplot as plt
import cv2

**Step 1: Download and extract the dataset**

Download the 'Dogs vs Cats' dataset manually from Kaggle or use the Kaggle API.

If you're using the Kaggle API, upload your `kaggle.json` file and run the following commands:

```python
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

!kaggle datasets download -d salader/dogs-vs-cats

import zipfile
with zipfile.ZipFile('/content/dogs-vs-cats.zip', 'r') as zip_ref:
    zip_ref.extractall('/content')


In [ ]:
# --- Step 2: Load datasets ---

train_ds = keras.utils.image_dataset_from_directory(
    directory='/content/train',  # Change path to your train folder
    labels="inferred",
    label_mode="int",
    batch_size=32,
    image_size=(256, 256)
)

validation_ds = keras.utils.image_dataset_from_directory(
    directory='/content/test',  # Change path to your test/validation folder
    labels="inferred",
    label_mode="int",
    batch_size=32,
    image_size=(256, 256)
)

In [ ]:
# --- Step 3: Normalize images ---

def normalize(image, label):
    image = tf.cast(image / 255., tf.float32)
    return image, label

train_ds = train_ds.map(normalize)
validation_ds = validation_ds.map(normalize)

In [ ]:
## --- Step 4: Build CNN model ---
# 3 convolutional layer
# first layer has 32 filters, 2nd layer has 64 filters and 3rd layer has 128 filters
model = Sequential()

model.add(Conv2D(32, kernel_size=(3,3), padding='valid', activation='relu', input_shape=(64,64,3)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2), strides=2, padding='valid'))

model.add(Conv2D(64, kernel_size=(3,3), padding='valid', activation='relu', input_shape=(64,64,3)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2), strides=2, padding='valid'))

model.add(Conv2D(128, kernel_size=(3,3), padding='valid', activation='relu', input_shape=(64,64,3)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2), strides=2, padding='valid'))

model.add(GlobalAveragePooling2D())

#Fully connected layer
# 3 layers
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(1, activation='sigmoid'))

model.summary()

In [ ]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
# --- Step 5: Train the model ---
history = model.fit(
    train_ds,
    epochs=25,
    validation_data=val_ds,
    callbacks=[EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)]
)


In [ ]:
# Direct Evaluation (no model loading needed)
test_loss, test_acc = model.evaluate(test_ds)
print(f"\nTest Accuracy: {test_acc*100:.2f}%")


In [1]:
# --- Step 6: Plot training results ---

plt.figure(figsize=(8, 5))
plt.plot(history.history['accuracy'], label='Training Accuracy', color='blue', linewidth=2) # Changed cnn_history to historty
plt.plot(history.history['val_accuracy'], label='Validation Accuracy', color='red', linewidth=2) # Changed cnn_history to historty
plt.title('Training vs Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.grid(True, linestyle='--')

plt.figure(figsize=(14, 5))
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Training Loss', color='blue', linewidth=2)
plt.plot(history.history['val_loss'], label='Validation Loss', color='red', linewidth=2)
plt.title('Training vs Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.grid(True, linestyle='--')

plt.tight_layout()
plt.show()

0 for cat and 1 for dog

In [ ]:
# --- Step 7: Predict on a new image ---
# Load and preprocess an example image (change the path accordingly)
test_img_path = '/content/dog.jpg'
test_img = cv2.imread(test_img_path)
test_img = cv2.cvtColor(test_img, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB for plt.imshow
test_img_resized = cv2.resize(test_img, (256, 256))
test_input = test_img_resized.reshape((1, 256, 256, 3)) / 255.0  # Normalize input

plt.imshow(test_img_resized)
plt.title('Test Image')
plt.axis('off')
plt.show()

# Make prediction
prediction = model.predict(test_input)

# Classify result
if prediction[0][0] >= 0.5:
    print(f"Prediction: Dog ({prediction[0][0]:.4f})")
else:
    print(f"Prediction: Cat ({prediction[0][0]:.4f})")
